In [ ]:
'''
(C) Renata Wong 2023

Qiskit code for testing fidelity of derandomised classical shadow on the ground state energy of molecules.

Procedure:
1. Derandomize the molecule-in-question's Hamiltonian.
2. Choose a variational ansatz with initial parameters selected at random.
3. Apply the derandomized Hamiltonian as basis change operators to the ansatz.
4. Measure the ansatz in the Pauli Z basis and store the results as a shadow.
5. Obtain the expectation value of the molecular Hamiltonian from the shadow.
6. Optimize for minimum Hamiltonian expectation value. 
7. Feed the calculated angles/parameters back to the ansatz.
8. Repeat steps 3-7 till the optimization is completed. 
9. Output the minimized expectation value of the molecular Hamiltonian and the mean-square-root-error. 

Note: Below we perform calculations on the molecular Hamiltonian of H_2.
To perform calculations on other molecules, you will need to specify their geometry, charge and spin 
to replace the values in the driver. 
'''

import numpy as np

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)


problem = driver.run()
hamiltonian = problem.hamiltonian

# The electronic Hamiltonian of the system
second_q_op = hamiltonian.second_q_op()

# Solving the electronic structure problem = determine the ground state energy of the molecule
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.mappers import BravyiKitaevMapper

# The Bravyi-Kitaev repserentation of the Fermionic Hamiltonian
mapper = BravyiKitaevMapper()
bkenc_hamiltonian = mapper.map(second_q_op)

print(bkenc_hamiltonian)

In [ ]:
'''
Format Hamiltonian terms and coefficients as required by the package predicting-quantum-properties
'''

hamiltonian_terms = []
weights = []

for observable in bkenc_hamiltonian:
    
    observable_str = str(observable)
    observable_str_clean = observable_str.strip()  # removes white spaces
    pauli_str_list = observable_str_clean.split('*')
    tuple_list = []
    
    for op_index, pauli_op in enumerate(pauli_str_list[1]):
        if pauli_op == 'I' or pauli_op == 'X' or pauli_op == 'Y' or pauli_op == 'Z':
            tuple_list.append((pauli_op, op_index-1))
    if len(tuple_list) > 0:
        hamiltonian_terms.append(tuple_list)
        weights.append(float(pauli_str_list[0].strip()))

system_size = len(hamiltonian_terms[0])

print(hamiltonian_terms)
print(weights)

In [ ]:
'''
Choose a variational ansatz.
Note that for molecules other than H_2 you may need to specify a different number of reps.
'''

from qiskit.circuit.library import EfficientSU2

reps = 1   
ansatz = EfficientSU2(4, su2_gates=['rx', 'ry'], entanglement='circular', reps=reps, skip_final_rotation_layer=True)  
    
ansatz.decompose().draw('mpl')

In [ ]:
'''
Generate derandomized Hamiltonian
'''

from derand.data_acquisition_shadow import derandomized_classical_shadow
from derand.prediction_shadow import estimate_exp

num_obs_evals = 67      # This number is selected in order to give ca. 1K Pauli operators. 

derandomized_hamiltonian = derandomized_classical_shadow(hamiltonian_terms, 
                                                                num_obs_evals, system_size, weight=weights)

print(derandomized_hamiltonian)

In [ ]:
'''
Define the cost function
'''

from qiskit_aer import QasmSimulator
backend = QasmSimulator(method='statevector', shots=1)

from qiskit import QuantumCircuit, execute

# Create circuit with just the randomised basis change operators
def rand_meas_circuit(pauli_op):
    rand_meas = QuantumCircuit(ansatz.num_qubits)
    for idx, op in enumerate(pauli_op):
        if op == 'X':
            rand_meas.x(idx)
        elif op == 'Y':
            rand_meas.y(idx)
        elif op == 'Z':
            rand_meas.z(idx)
    return rand_meas


def objective_function(params):
    """Compares the output distribution of our circuit with
    parameters `params` to the target distribution."""
    
    global HAMILTONIAN_EXPVAL
    
    # Assign parameters to the ansatz and simulate it
    # Generate circuits to measure random Paulis, one circuit for each Pauli
    
    shadow = []
    for pauli_op in derandomized_hamiltonian:
        
        qc = ansatz.bind_parameters(params)
        qc.compose(rand_meas_circuit(pauli_op))
        qc.measure_all()
        result = execute(qc, backend, shots=1).result()
        counts = result.get_counts()
        
        
        # We perform one single shot => index(1)
        # store the shadow in the form [[(Z,1),(Z,-1)...], [(Y,-1),(X,-1),...]] where inner list = snapshot
        # Because measurement output in Qiskit gives us states and not eigenvalues, we need to convert 0->1 and 1->-1
        
        output_str = list(list(counts.keys())[list(counts.values()).index(1)])
        output = [int(i) for i in output_str]
        eigenvals = [x+1 if x == 0 else x-2 for x in output]
        snapshot = [(op, eigenval) for op, eigenval in zip(pauli_op, eigenvals)]
        shadow.append(snapshot)
      
    
    # Now, we want to get the expectation values for the Hamiltonian from the shadow using the function
    # estimate_exp(full_measurement, one_observable)
    # where full_measurement = shadow and one_observable is any term in the Hamiltonian
    # total_exp_val contains the total expectation value of the Hamiltonian
    
    total_exp_val = 0.0
    for term, weight in zip(hamiltonian_terms, weights):
        exp_val, number_of_matches = estimate_exp(shadow, term)
        total_exp_val = total_exp_val + (weight * exp_val)
    
    
    # Calculate the cost as the distance between the 
    # distribution and the target distribution
    
    cost = abs(HAMILTONIAN_EXPVAL - total_exp_val)
    if total_exp_val < HAMILTONIAN_EXPVAL:
        HAMILTONIAN_EXPVAL = total_exp_val
    
    return cost


In [ ]:
'''
Classical optimisation step
'''

# Counter for the execution time
import time
start_time = time.time()

HAMILTONIAN_EXPVAL = 100.0


# Classical optimizer
from qiskit.algorithms.optimizers import SLSQP
optimizer = SLSQP(maxiter=500)   

# Create the initial parameters
params = np.random.rand(ansatz.num_parameters)
result = optimizer.minimize(fun=objective_function, x0=params, args=randomized_hamiltonian)


print("HAMILTONIAN_EXPVAL = ", HAMILTONIAN_EXPVAL)

elapsed_time = time.time() - start_time
print("Execution time = ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
'''
Calculate the error.
Note that if you run the experiment multiple times, you will need to sum over (EXPECTED_EIGENVALUE - HAMILTONIAN_EXPVAL)**2
and divide by the number of experiments. 
'''

EXPECTED_EIGENVALUE = -1.86

rmse_derandomised_cs = np.sqrt((EXPECTED_EIGENVALUE - HAMILTONIAN_EXPVAL)**2)
print(f"The root-mean-squared error for derandomized classical shadow on H2: {rmse_derandomised_cs}")


In [ ]:
'''
Above we have assumed that the ground state energy of H_2 is ca. -1.86.
Below we corroborate this assumption using a classical minimum eigensolver on our Hamiltonian.
'''

from qiskit_nature.second_q.mappers import BravyiKitaevMapper, QubitConverter
converter = QubitConverter(BravyiKitaevMapper())

from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
numpy_solver = NumPyMinimumEigensolver()   

calc = GroundStateEigensolver(converter, numpy_solver)
res = calc.solve(problem)
print('Electronic ground state energy:\n', res) 